In [2]:
import pickle
import torch
import json
import numpy as np
from data import MovieLensDataLoader

In [3]:
# outputs_gru_dev = torch.load('dumps/preds-rnn-val-20210722060645.pt')
# outputs_gru_test = torch.load('dumps/preds-rnn-test-20210722060654.pt')
# outputs_bert_dev = torch.load('dumps/preds-val-20210722061029.pt')
# outputs_bert_test = torch.load('dumps/preds-test-20210722061114.pt')
outputs_gru_dev = torch.load('dumps/preds-rnn-val-20210722200609.pt')
outputs_gru_test = torch.load('dumps/preds-rnn-test-20210722200616.pt')
outputs_bert_dev = torch.load('dumps/preds-val-20210722200313.pt')
outputs_bert_test = torch.load('dumps/preds-test-20210722200338.pt')
len(outputs_gru_dev)

5337

In [4]:
dm = MovieLensDataLoader(
    data_dir='datasets/MovieLens-1M-5Star',
    model_type='BERT',
    model_name_or_path='bert-base-cased',
    max_seq_length=128,
    min_session_length=7,
    max_session_length=7
)
dm.setup()
id2name = json.load(open(f'datasets/MovieLens-1M-5Star/id2name.json'))

Loading features from cached dir datasets/MovieLens-1M-5Star/cached_bert-base-cased_28996_128_7_7_10
DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'ids', 'input_ids', 'input_str', 'label', 'mask_idxs', 'token_type_ids', 'user'],
        num_rows: 21348
    })
    val: Dataset({
        features: ['attention_mask', 'ids', 'input_ids', 'input_str', 'label', 'mask_idxs', 'token_type_ids', 'user'],
        num_rows: 5337
    })
    test: Dataset({
        features: ['attention_mask', 'ids', 'input_ids', 'input_str', 'label', 'mask_idxs', 'token_type_ids', 'user'],
        num_rows: 5337
    })
})


In [5]:
def compute_metrics(logits, labels, prefix):
    top_idxs = (-logits).argsort()
    r_20 = (top_idxs[:, :20] == labels.reshape(-1, 1)).sum(-1).mean(-1)
    return {f'{prefix}/r@20': r_20}

In [15]:
outputs = outputs_bert_test
compute_metrics(logits=np.stack([item['logits'] for item in outputs]), labels=np.array([item['labels'] for item in outputs]), prefix='val')

{'val/r@20': 0.00018737118231216038}

In [7]:
outputs_gru_dev[0], outputs_bert_dev[0]

({'logits': array([  3.3398957,  -4.20791  ,  -2.7382689, ...,  -5.5328903,
         -13.28255  ,  -1.2705415], dtype=float32),
  'labels': 2286},
 {'logits': array([43.743164, 50.259804, 50.728096, ..., 53.689045, 57.532642,
         49.169582], dtype=float32),
  'labels': 2286})

In [24]:
idx = 20
K = 10

label = int(dm.dataset['val']['label'][idx])
input_str = dm.dataset['val']['input_str'][idx]
user = int(dm.dataset['val']['user'][idx])
print(user)
print(input_str, '\n')

print('------------BERT--------------')

outputs = outputs_bert_dev
label_verify = outputs[idx]['labels']
assert(label == label_verify)
logits = outputs[idx]['logits']
top_idxs = logits.argsort().tolist()
for id in top_idxs[:K]:
    print(id, id2name[id], logits[id])
print()
print(label, id2name[label], logits[label], top_idxs.index(label) + 1)

print('------------GRU--------------')

outputs = outputs_gru_dev
label_verify = outputs[idx]['labels']
assert(label == label_verify)
logits = outputs[idx]['logits']
top_idxs = (-logits).argsort().tolist()
for id in top_idxs[:K]:
    print(id, id2name[id], logits[id])
print()
print(label, id2name[label], logits[label], top_idxs.index(label) + 1)

25
A user watched Bicentennial Man, Face/Off, Back to the Future Part II, Indiana Jones and the Last Crusade, Transformers: The Movie, The. Now the user may want to watch[MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK] 

------------BERT--------------
1250 Back to the Future 44.068634
2559 Star Wars: Episode I - The Phantom Menace 44.491608
1187 Transformers: The Movie, The 44.988358
3038 Backdraft 45.034927
1942 Back to the Future Part II 45.21419
1192 Star Wars: Episode VI - Return of the Jedi 45.316505
2502 The Matrix 45.42791
257 Star Wars: Episode IV - A New Hope 45.430084
1183 The Good, the Bad and the Ugly 45.517357
847 The Godfather 45.660126

2432 October Sky 51.56845 1455
------------GRU--------------
2046 Indiana Jones and the Temple of Doom 4.0144587
770 Independence Day 3.4195216
2012 The Little Mermaid 3.2585962
606 Heavy Metal 3.232671
2559 Star Wars: Episode I - The Phantom Menace 3.1830375
2124 Willow 3.1431715
1943 Back to the Future Part III 3.1429465
133